In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install scikit-learn
!pip install scikit-learn-extra
!pip install nltk
!pip install path

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk

# Punkt é necessário para realizar a tokenização de palavras e sentenças
nltk.download('punkt') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sklearn_extra.cluster import KMedoids
import numpy as np
import re
from path import Path

documento  = []
notas      = []
diretorio  = Path('/content/drive/MyDrive/Proj_PFC/Corpus_TXT_PFC2/Hotel4/')
nomesumario = diretorio.split('/')

# Usa a pasta de onde os documentos foram retirados como nome do sumário
nomesumario = nomesumario[len(nomesumario)-2]
#print(nomesumario)

# Carrega documentos para serem sumarizados. Entretanto, os testes iniciais
# são feitos com sentenças pré-selecionadas, pois as técnicas Opizer-E e Tadano
# trabalham dessa forma em suas implementações por Condori
for arquivos in diretorio.files('*.txt'):
    with arquivos.open(mode='rt', encoding='utf-8') as leitor:
        documento = leitor.readlines()

# Vetor de nota previamente utilizado para verificar a polaridade das sentenças.
# Não será utilizado até então pois o foco do LexRank é em extrair sentenças sem
# verificar a polaridade.
#for arquivos in diretorio.files('*.score'):
    #with arquivos.open(mode='rt', encoding='utf-8') as leitor:
        #notas = leitor.readlines()


#print(documento)

# Salva as opinioes em um formato lista de listas, para separar sentenças por
# opiniões e facilitar a classificação de polaridade futuramente, além de tornar
# a representação visual delas mais concisa durante o processamento.
opinioes_temp = []
opinioes = []
sentenca = 0

# Classifica as sentenças, sempre estando no índice 0 já que as sentenças 
# classificadas são removidas do documento inicial, portanto seu tamanho 
# diminui até 0
while sentenca < len(documento): 
    #print(sentenca, " | documento_len = ", len(documento) )
    #print(documento[sentenca])
    if(documento[sentenca] == '---\n'): # Se essa linha for um delimitador
      documento.remove(documento[sentenca]) # Remova o delimitador
      opinioes.append(opinioes_temp) # Adicione a opinião à lista principal
      opinioes_temp = [] # Esvazia o vetor de sentenças da opinião processada
    else: # Se não for um delimitador
      opinioes_temp.append(documento[sentenca]) # Adicione a sentença à opinião
      documento.remove(documento[sentenca]) # Remova-a do documento recebido

#print(opinioes)

#
# Pré-processamento textual do conteúdo passado como entrada
#

# Coloca todas as palavras em minúsculo para evitar repetição de tokens
for i in range(0, len(opinioes)):
  for j in range(0, len(opinioes[i])):
    opinioes[i][j] = opinioes[i][j].lower()

# Remove pontuações do texto, para evitar que estes símbolos se tornem tokens
for i in range(0, len(opinioes)):
  for j in range(0, len(opinioes[i])):
    opinioes[i][j] = re.sub(r'[^\w\s]', '', opinioes[i][j])

#print("Sentenças separadas por opinião:", opinioes, '\n')



#
# Passo 1: Tokenização de palavras
#
tokens_palavras = []
tokens_temp = []

# Cria uma lista de listas, onde cada lista interna representa os tokens de uma
# opinião como um todo, não fazendo separação por sentenças até o momento.
for i in range(0, len(opinioes)):
  for j in range(0, len(opinioes[i])):
    tokens_temp.extend(nltk.word_tokenize(opinioes[i][j]))
  tokens_palavras.append(tokens_temp)
  tokens_temp = []

#print("Tokens das sentenças:", tokens_palavras, '\n')



#
# Passo 2: Remoção de stopwords
#

# Carrega as stopwords do arquivo salvo na pasta de recursos
arq_stopwords = open('/content/drive/MyDrive/Proj_PFC/KMedoids/recursos/stopwords_pt.txt', 'rt')
stopwords = arq_stopwords.readlines()

# Remove caracteres de nova linha no final de cada stopword do arquivo
for i in range(0, len(stopwords)):
  stopwords[i] = stopwords[i].rstrip('\n')

#print("Stopwords:", stopwords, '\n')


tokens_sem_stopwords = []
stopword_temp = []

for i in range (0, len(tokens_palavras)):
  stopword_temp = []
  for palavra in tokens_palavras[i]:
    if not(palavra in stopwords):
        stopword_temp.append(palavra)
  tokens_sem_stopwords.append(stopword_temp)
  
#print("Tokens sem stopwords:", tokens_sem_stopwords, '\n')



#
# Passo 3: Cálculo de frequências por TF-IDF.
#
### Implementação do TF-IDF baseada em: https://www.computersciencemaster.com.br/como-implementar-o-tf-idf-em-python/

term_freqs = {}
inv_doc_freqs = {}

# Cálculo do Term Frequency
for i in range(0, len(tokens_sem_stopwords)):
  for palavra in tokens_sem_stopwords[i]:
    if palavra not in term_freqs.keys():
      term_freqs[palavra] = 1
    else:
      term_freqs[palavra] += 1

#print("TF das palavras:", term_freqs, '\n')


# Cálculo do Inverse Document Frequency

num_docs = 0

# Cada opinião é tratada como um documento, então incremente num_docs caso o 
# termo esteja presente num dos elementos do vetor de tokens a nível de opinião
for term in term_freqs:
  num_docs = 0

  for i in range(0, len(tokens_sem_stopwords)):
    if term in tokens_sem_stopwords[i]:
      num_docs += 1

  inv_doc_freqs[term] = np.log((len(tokens_sem_stopwords)/num_docs)+1)

#print("IDF das palavras:", inv_doc_freqs)


# Calculo da matriz TF para as palavras encontradas.


matriz_tf = {}
tf_doc = []
doc_frequency = 0
num_tokens = 0

for i in range(0, len(tokens_sem_stopwords)):
  num_tokens += len(tokens_sem_stopwords[i])

for term in term_freqs:
  tf_doc = []
  for i in range(0, len(tokens_sem_stopwords)):
    doc_frequency = 0
    for word in tokens_sem_stopwords[i]:
      if word == term:
        doc_frequency += 1
    tf_term = doc_frequency/num_tokens
    tf_doc.append(tf_term)

  matriz_tf[term] = tf_doc

#print("Matriz tf de tamanho[",len(matriz_tf),"]\n")


# Cálculo final do TF-IDF

score = 0
tf_idf = []
matriz_tf_idf = []

for term in matriz_tf.keys():
  score = 0
  tf_idf = []
  for value in matriz_tf[term]:
    score = value * inv_doc_freqs[term]
    tf_idf.append(score)
  matriz_tf_idf.append(tf_idf)
#print("Matriz tf_idf de tamanho[",len(matriz_tf_idf),"][",len(matriz_tf_idf[0]),"]\n")



#
#Passo 4: A ser realizado, caso seja necessário
#


In [ ]:
#
#
# Implementação do KMedoids
#
#

# kmedoids_temp é um arquivo usado para gravar dados de teste da execução do 
# algoritmo
#kmedoids_temp = open('/content/drive/MyDrive/Proj_PFC/KMedoids/temp/kmedoids_temp.txt', 'w')
num_clusters = 49 # Número máximo de agrupamentos e sentenças a ser processado
mat_sentencas = np.asarray(matriz_tf_idf)
no_match = 0
palavras_centrais = []
palavras_centrais_ind = []
sent_score = [] # [opinião][sentença]

for i in range(0, len(opinioes)):
  sent_score_temp = []
  for j in range(0, len(opinioes[i])):
    sent_score_temp.append(0)
  sent_score.append(sent_score_temp)

sumarizador = KMedoids(n_clusters=num_clusters, metric='euclidean', method='pam', init='heuristic', max_iter=5000, random_state=None).fit(mat_sentencas)
#print(sumarizador.cluster_centers_) # Medoids do agrupamento

#kmedoids_temp.write("Medoids:")
#kmedoids_temp.write(str(sumarizador.cluster_centers_))
#kmedoids_temp.write('\n')

# Encontra as palavras que compõem os medoids descobertos e as salva em índices
# específicos com base no índice do medoid
for i in range(0, len(sumarizador.cluster_centers_)):
  palavras_centrais_ind = []
  for j in range(0, len(matriz_tf_idf)):
    no_match = 0
    for k in range(0, len(matriz_tf_idf[j])):
      if matriz_tf_idf[j][k] != sumarizador.cluster_centers_[i][k]:
        no_match = 1
      
    if(no_match == 0):
      #print(matriz_tf.get(j))
      palavras_centrais_ind.append(list(matriz_tf.keys())[j])
  
  palavras_centrais.append(palavras_centrais_ind)

#print(palavras_centrais)

#kmedoids_temp.write("Palavras centrais de acordo com os medoids:")
#kmedoids_temp.write(str(palavras_centrais))
#kmedoids_temp.write('\n')

# Cruza essas palavras com as sentenças presentes nas opiniões passadas como 
# entrada e calcula a pontuação delas de acordo com a 

for i in range(0, len(palavras_centrais)):
  for j in range(0, len(opinioes)):
    for k in range(0, len(opinioes[j])):
      if any(word in opinioes[j][k] for word in palavras_centrais[i]):
      #if word in palavras_centrais[i][j] in opinioes[k]:
        sent_score[j][k] += 1
      #print(opinioes[j][k])

#print("Pontuações das sentenças de cada opinião:", sent_score)

#kmedoids_temp.write("Pontuação das sentenças de acordo com o kmedoids:")
#kmedoids_temp.write(str(sent_score))
#kmedoids_temp.write('\n')

# Pega o índice da sentença com maior número de pontos de cada opinião para 
# adicionar ao sumário final

sentencas_centrais = []
indice_sentencas_centrais = []

for i in range(0, len(sent_score)):
  sentencas_centrais.append(0)
  indice_sentencas_centrais.append(0)
  for j in range(0, len(sent_score[i])):
    if sent_score[i][j] > sentencas_centrais[i]:
      sentencas_centrais[i] = sent_score[i][j]
      indice_sentencas_centrais[i] = j

#print("Índice das sentenças centrais:", indice_sentencas_centrais)

# Gera o sumário final

#sumario_pos   = []
#sumario_neg   = []
sumario_final = []

for i in range(0, num_clusters):
  sumario_final.append(opinioes[i][indice_sentencas_centrais[i]])

### Classificação de polaridade ainda não será contemplada!!!
#sumario_final.append("Opiniões positivas:\n\n")
#for i in range(0, len(opinioes) ):
#  if(int(notas[i]) > 3):
#    sumario_final.append(opinioes[i][indice_sentencas_centrais[i]])
#    sumario_final.append('\n')

#sumario_final.append("\n\nOpiniões negativas:\n\n")
#for i in range(0, len(opinioes) ):
#  if(int(notas[i]) < 3):
#    sumario_final.append(opinioes[i][indice_sentencas_centrais[i]])
#    sumario_final.append('\n')

#sumario_final.append("\n\nOpiniões neutras:\n\n")
#for i in range(0, len(opinioes) ):
#  if(int(notas[i]) == 3):
#    sumario_final.append(opinioes[i][indice_sentencas_centrais[i]])
#    sumario_final.append('\n')

arq_sum = open('/content/drive/MyDrive/Proj_PFC/KMedoids/Sumarios_PFC2/' + nomesumario + '_KMedoids.sum', 'w')

for i in range(0, len(sumario_final)):
  arq_sum.write(sumario_final[i])

#sumarizador.predict([[0,1], [1,2]])